In [11]:
from glob import glob
import pandas as pd
import numpy as np
from scipy import stats  
import matplotlib.pyplot as plt
import japanize_matplotlib

In [12]:
files = glob("../csv/演習問題_データアナリティクス基礎/*")
for i, f in enumerate(files):
    print(i, f)


0 ../csv/演習問題_データアナリティクス基礎\第10章演習問題_乱数で発生させて分布の基本統計量.xlsx
1 ../csv/演習問題_データアナリティクス基礎\第2章演習問題_クレームデータ.xlsx
2 ../csv/演習問題_データアナリティクス基礎\第3章演習問題_職場ストレス.xlsx
3 ../csv/演習問題_データアナリティクス基礎\第4章演習問題_児童のネット利用時間.xlsx
4 ../csv/演習問題_データアナリティクス基礎\第5章演習問題_生鮮野菜の購入額.xlsx
5 ../csv/演習問題_データアナリティクス基礎\第7章演習問題_授業効果の検証.xlsx
6 ../csv/演習問題_データアナリティクス基礎\第8章演習問題1_健康食品・化粧品販売額.xlsx
7 ../csv/演習問題_データアナリティクス基礎\第8章演習問題2_百貨店の衣料品売上高.xlsx
8 ../csv/演習問題_データアナリティクス基礎\第9章演習問題_自由記述データ＆形態素解析データ.xlsx


In [13]:
df = pd.read_excel(files[6])
print(df.columns)
df.columns = ['year', 'food', 'care']
df.info()
df

Index(['年', '健康食品', 'ビューティケア\n（化粧品・小物）'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   year    7 non-null      int64
 1   food    7 non-null      int64
 2   care    7 non-null      int64
dtypes: int64(3)
memory usage: 296.0 bytes


,year,food,care
0,2014,164669,726156
1,2015,190617,811167
2,2016,197031,852185
3,2017,206730,910175
4,2018,217745,963666
5,2019,221759,1008208
6,2020,226388,903560


In [14]:
df = df.set_index('year')
df


,food,care
year,,
2014,164669,726156
2015,190617,811167
2016,197031,852185
2017,206730,910175
2018,217745,963666
2019,221759,1008208
2020,226388,903560


健康食品に関する2014年を基準とした各年の指数
$$ index = \frac{Comparison}{Reference} $$

In [15]:
index = df['food'] / df.loc[2014, 'food'] * 100
index

year
2014    100.000000
2015    115.757671
2016    119.652758
2017    125.542755
2018    132.231932
2019    134.669549
2020    137.480643
Name: food, dtype: float64

2014年基準のビューティケア商品の販売額増加率
- 増加率・成長率(％) = (比較の値 – 比較前の値) / 比較前の値 × 100

$$ Ratio \ of \ increase = \frac{Comparison - Reference}{Reference} $$


In [16]:
reference = 2014
target = 'care'
(df[target] - df.loc[reference, target]) / df.loc[reference, target] * 100

year
2014     0.000000
2015    11.706989
2016    17.355637
2017    25.341524
2018    32.707848
2019    38.841792
2020    24.430563
Name: care, dtype: float64

2017年基準のビューティケア商品の販売額増加率

In [17]:
reference = 2017
target = 'care'
(df[target] - df.loc[reference, target]) / df.loc[reference, target] * 100

year
2014   -20.217980
2015   -10.877908
2016    -6.371302
2017     0.000000
2018     5.877002
2019    10.770786
2020    -0.726783
Name: care, dtype: float64

ビューティケア商品の対前年増加率の最大値

In [22]:
df.index[1:]

Int64Index([2015, 2016, 2017, 2018, 2019, 2020], dtype='int64', name='year')

In [24]:
df_yoy = pd.DataFrame([df['care'][1:].values, df['care'][:-1].values]).T
df_yoy.columns = ['comparison', 'reference']
df_yoy['increase_rete'] = (df_yoy['comparison'] - df_yoy['reference']) / df_yoy['reference'] * 100
df_yoy.loc[df_yoy['increase_rete'].idxmax(), 'mark'] = 'max'
df_yoy.loc[df_yoy['increase_rete'].idxmin(), 'mark'] = 'min'
df_yoy.index = df.index[1:]
df_yoy

,comparison,reference,increase_rete,mark
year,,,,
2015,811167,726156,11.706989,max
2016,852185,811167,5.056665,NaN
2017,910175,852185,6.804860,NaN
2018,963666,910175,5.877002,NaN
2019,1008208,963666,4.622141,NaN
2020,903560,1008208,-10.379604,min


健康食品の成長率について2018年と2019年を比較する
$$ growth \ rate = \frac{This \ year - Target \ year}{Target \ year} $$

In [19]:
targets = [2018, 2019]
rates = []
for t in targets:
    rate = (df.loc[t, 'food'] - df.loc[t-1, 'food']) / df.loc[t-1, 'food'] * 100
    rates.append(rate)
print(f'{rates[0] - rates[1]:.01f}')

3.5
